In [1]:
import re
from pathlib import Path

import numpy as np

In [2]:
input = Path("example.txt").read_text()
print(input)

Button A: X+94, Y+34
Button B: X+22, Y+67
Prize: X=8400, Y=5400

Button A: X+26, Y+66
Button B: X+67, Y+21
Prize: X=12748, Y=12176

Button A: X+17, Y+86
Button B: X+84, Y+37
Prize: X=7870, Y=6450

Button A: X+69, Y+23
Button B: X+27, Y+71
Prize: X=18641, Y=10279



In [3]:
cost = np.array([3, 1], dtype=np.int64)
print(cost)

[3 1]


In [4]:
data = re.findall(
    r"Button A: X\+(?P<A_00>\d*), Y\+(?P<A_01>\d*)\n"
    r"Button B: X\+(?P<A_10>\d*), Y\+(?P<A_11>\d*)\n"
    r"Prize: X=(?P<b_0>\d*), Y=(?P<b_1>\d*)",
    input,
)
data

[('94', '34', '22', '67', '8400', '5400'),
 ('26', '66', '67', '21', '12748', '12176'),
 ('17', '86', '84', '37', '7870', '6450'),
 ('69', '23', '27', '71', '18641', '10279')]

In [5]:
equations = np.array(data, dtype=np.int64).reshape((-1, 3, 2))
A = np.array([a.T for a in equations[:, :2]])
b = equations[:, 2]
print(A)

[[[94 22]
  [34 67]]

 [[26 67]
  [66 21]]

 [[17 84]
  [86 37]]

 [[69 27]
  [23 71]]]


In [6]:
print(b)

[[ 8400  5400]
 [12748 12176]
 [ 7870  6450]
 [18641 10279]]


In [7]:
# Solution 1
X = np.linalg.solve(A, b).round().astype(np.int64)
dot_product = [A_i @ X_i for X_i, A_i in zip(X, A)]
is_valid = np.isclose(dot_product, b).all(axis=1)
is_under_100 = (X <= 100).all(axis=1)

button_presses = X[is_valid & is_under_100]

np.sum(button_presses * cost)

480

In [8]:
# Solution 2
ROUNDING_ERROR = 10000000000000
X = np.linalg.solve(A, b + ROUNDING_ERROR).round().astype(np.int64)

is_valid = np.isclose([A_i @ X_i - ROUNDING_ERROR for A_i, X_i in zip(A, X)], b).all(
    axis=1
)

button_presses = X[is_valid]

np.sum(button_presses * cost)

875318608908